In [1]:
import os

In [2]:
# install youtube-dl using pip
%pip install youtube-dl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
file = "Telugu_trailer.mp4"
# if file doesnt exist, download from https://www.youtube.com/watch?v=7kpGQ_hbU30
if not os.path.exists(file):
	%youtube-dl -f 'bestvideo[ext=mp4]' --output "Telugu_trailer.%(ext)s" "https://www.youtube.com/watch?v=7kpGQ_hbU30"

UsageError: Line magic function `%youtube-dl` not found.
